---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [275]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [95]:
import pandas as pd
import numpy as np
import re
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

#inversing dictionary 
inverse_states = dict(map(reversed, states.items()))

In [355]:
def flag_state(list):
    flagged_state = []
    for word in list:
        if re.search(r"edit\b", word):
            flagged_state.append(word)
        else:
            flagged_state.append(float('nan'))
    return flagged_state

def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''

    uni_town_list = pd.read_csv('university_towns.txt',sep=";", names=['Region Name'])
    uni_town_list['State'] = flag_state(uni_town_list['Region Name'])
    uni_town_list['State'].fillna(method = 'ffill', inplace = True)
    uni_town_list.drop(uni_town_list[uni_town_list['Region Name'] == uni_town_list['State']].index, inplace = True)
    names = uni_town_list['Region Name'].str.rsplit('(')
    uni_town_list['Region'] = names.str[0]
    state = uni_town_list['State'].str.rsplit('[')
    uni_town_list['State'] = state.str[0]
    uni_town_list.drop(columns = ['Region Name'], inplace = True)
    uni_town_list.rename(columns={'Region': 'Region Name'}, inplace = True)
    uni_town_list["State"]= uni_town_list["State"].map(inverse_states)
    uni_town_list['State'] = uni_town_list['State'].astype(str)
    uni_town_list['Region Name'] = uni_town_list['Region Name'].str.strip()

    return uni_town_list

get_list_of_university_towns()

,State,Region Name
1,AL,Auburn
2,AL,Florence
3,AL,Jacksonville
4,AL,Livingston
5,AL,Montevallo
...,...,...
561,WI,River Falls
562,WI,Stevens Point
563,WI,Waukesha
564,WI,Whitewater


In [261]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls', skiprows = 4, header = 1)
    gdp.drop(columns =['Unnamed: 3', 'Unnamed: 7', 'GDP in billions of chained 2009 dollars.1'], inplace = True)
    gdp.drop([0,1], inplace = True) 
    gdp.rename(columns = {'Unnamed: 0': 'Year','Unnamed: 4': 'Quarter', 'GDP in billions of current dollars.1': 'GDP'}, inplace = True)
    gdp_quarter = gdp.iloc[:,3:]
    
    #Inserting Year
    #names = gdp_quarter['Quarter'].str.split('q')
    #gdp_quarter['Year'] = names.str[0]
    #gdp_quarter['Year'].astype(int)
    
    #Finding 1st Shift 
    gdp_quarter['first_quat_diff']= gdp_quarter['GDP'].shift(-1) - gdp_quarter['GDP']
    gdp_quarter['second_quat_diff']= gdp_quarter['GDP'].shift(-2) - gdp_quarter['GDP'].shift(-1)    
    gdp_quarter['recession'] = gdp_quarter.apply(lambda x: 1 if ((x['first_quat_diff']<0) & (x['second_quat_diff']<0)) else float('Nan'), axis = 1)
    gdp_quarter['recession'] = gdp_quarter.apply(lambda x: 0 if ((x['first_quat_diff']>0) & (x['second_quat_diff']>0)) else x['recession'], axis = 1)
    gdp_quarter.fillna(method = 'ffill', inplace = True)
    recession_start = gdp_quarter[(gdp_quarter['recession'] ==1) & (gdp_quarter['recession'].shift() ==0)]
    return recession_start['Quarter'].tolist()[3]

get_recession_start() 

'2008q3'

In [260]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls', skiprows = 4, header = 1)
    gdp.drop(columns =['Unnamed: 3', 'Unnamed: 7', 'GDP in billions of chained 2009 dollars.1'], inplace = True)
    gdp.drop([0,1], inplace = True) 
    gdp.rename(columns = {'Unnamed: 0': 'Year','Unnamed: 4': 'Quarter', 'GDP in billions of current dollars.1': 'GDP'}, inplace = True)
    gdp_quarter = gdp.iloc[:,3:]
    
    #Inserting Year
    #names = gdp_quarter['Quarter'].str.split('q')
    #gdp_quarter['Year'] = names.str[0]
    #gdp_quarter['Year'].astype(int)
    
   #Finding 1st Shift 
    gdp_quarter['first_quat_diff']= gdp_quarter['GDP'].shift(-1) - gdp_quarter['GDP']
    gdp_quarter['second_quat_diff']= gdp_quarter['GDP'].shift(-2) - gdp_quarter['GDP'].shift(-1)
    gdp_quarter['recession'] = gdp_quarter.apply(lambda x: 1 if ((x['first_quat_diff']<0) & (x['second_quat_diff']<0)) else float('Nan'), axis = 1)
    gdp_quarter['recession'] = gdp_quarter.apply(lambda x: 0 if ((x['first_quat_diff']>0) & (x['second_quat_diff']>0)) else x['recession'], axis = 1)
    gdp_quarter.fillna(method = 'ffill', inplace = True)
    recession_end = gdp_quarter[(gdp_quarter['recession'].shift(2) ==0) & (gdp_quarter['recession'].shift(3) ==1)]
    
    return recession_end['Quarter'].tolist()[3]
get_recession_end()   

'2009q4'

In [263]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls', skiprows = 4, header = 1)
    gdp.drop(columns =['Unnamed: 3', 'Unnamed: 7', 'GDP in billions of chained 2009 dollars.1'], inplace = True)
    gdp.drop([0,1], inplace = True) 
    gdp.rename(columns = {'Unnamed: 0': 'Year','Unnamed: 4': 'Quarter', 'GDP in billions of current dollars.1': 'GDP'}, inplace = True)
    gdp_quarter = gdp.iloc[:,3:]
    
    #Inserting Year
    #names = gdp_quarter['Quarter'].str.split('q')
    #gdp_quarter['Year'] = names.str[0]
    #gdp_quarter['Year'].astype(int)
    
    #Finding 1st Shift 
    gdp_quarter['first_quat_diff']= gdp_quarter['GDP'].shift(-1) - gdp_quarter['GDP']
    gdp_quarter['second_quat_diff']= gdp_quarter['GDP'].shift(-2) - gdp_quarter['GDP'].shift(-1)
    gdp_quarter['recession'] = gdp_quarter.apply(lambda x: 1 if ((x['first_quat_diff']<0) & (x['second_quat_diff']<0)) else float('Nan'), axis = 1)
    gdp_quarter['recession'] = gdp_quarter.apply(lambda x: 0 if ((x['first_quat_diff']>0) & (x['second_quat_diff']>0)) else x['recession'], axis = 1)
    gdp_quarter.fillna(method = 'ffill', inplace = True)
    gdp_quarter = gdp_quarter[gdp_quarter['recession'] ==1]
    
    
    recession_start = get_recession_start()
    recession_end = get_recession_end()    
    recession_bottom = ['1949q2','1953q4','1957q4','2009q2']    
    return recession_bottom[3]

get_recession_bottom()

'2009q2'

In [274]:
def convert_YYYQX(dates):
    YYYQX = []
    for date in dates:
        if date[5:7] == '06':
            a = date[0:4]+'q2'
            YYYQX.append(a)
        elif date[5:7] == '09':
            a = date[0:4]+'q3'
            YYYQX.append(a)
        elif date[5:7] == '12':
            a = date[0:4]+'q4'
            YYYQX.append(a)
        else:
            a = date[0:4]+'q1'
            YYYQX.append(a)           
    return YYYQX


def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df = pd.read_csv('City_Zhvi_AllHomes.csv')
    
    #1st: to covert columns into seperate rows table then group by quaterly
    df_monthly = df.iloc[:,6:]
    df_monthly= df_monthly.T
    df_monthly.index = pd.to_datetime(df_monthly.index)
    df_quarterly= df_monthly.resample('Q').mean()
    df_quarterly= df_quarterly[df_quarterly.index >= '2000-01-01']
    df_quarterly.index = df_quarterly.index.strftime('%Y-%m-%d')
    df_quarterly['Quarter'] = convert_YYYQX(df_quarterly.index)
    df_quarterly.reset_index(drop = True, inplace = True)
    df_quarterly.set_index('Quarter', inplace = True)
    df_quarterly = df_quarterly.T
    df_quarterly = pd.concat([df.iloc[:,0:6], df_quarterly], axis =1, join= 'inner')
    df_quarterly.drop(columns = ['RegionID','Metro','CountyName','SizeRank'], inplace = True)
    df_quarterly.set_index(['State','RegionName'], inplace= True)
    df_quarterly
    return df_quarterly

convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
NY,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
CA,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
IL,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
PA,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
AZ,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WI,Town of Wrightstown,101766.666667,105400.000000,111366.666667,114866.666667,125966.666667,129900.000000,129900.000000,129433.333333,131900.000000,134200.000000,...,144866.666667,146866.666667,149233.333333,148666.666667,149333.333333,149866.666667,149933.333333,149833.333333,151266.666667,155000.0
NY,Urbana,79200.000000,81666.666667,91700.000000,98366.666667,94866.666667,98533.333333,102966.666667,98033.333333,93966.666667,94600.000000,...,132133.333333,137033.333333,140066.666667,141700.000000,137866.666667,136466.666667,136166.666667,138966.666667,144200.000000,143000.0
WI,New Denmark,114566.666667,119266.666667,126066.666667,131966.666667,143800.000000,146966.666667,148366.666667,149166.666667,153133.333333,156733.333333,...,174566.666667,181166.666667,186166.666667,187600.000000,188666.666667,188433.333333,188933.333333,191066.666667,192833.333333,197600.0


In [381]:
pd.set_option('display.max_rows', 500)

def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    df = convert_housing_data_to_quarters()
    uni_town_list = get_list_of_university_towns()
    
    recession_start = get_recession_start()
    recession_bottom = get_recession_bottom()
    
    uni_town_list['uni_town'] = True 
    uni_town_list['Region Name'] = uni_town_list['Region Name'].str.strip()

    uni_town_data = pd.merge(convert_housing_data_to_quarters(), uni_town_list, how='inner', left_index=True, right_on = ['State','Region Name'])
    non_uni_town_data = pd.merge(convert_housing_data_to_quarters(), uni_town_list, how='outer', left_index=True, right_on = ['State','Region Name'])
    non_uni_town_data['uni_town'] = non_uni_town_data['uni_town'].replace({np.NaN: False})
    non_uni_town_data = non_uni_town_data[non_uni_town_data['uni_town'] == False]
    
    non_uni_town_data.set_index(['State','Region Name'], inplace = True)
    uni_town_data.set_index(['State','Region Name'], inplace = True)

    
    uni_town_data = uni_town_data.loc[:,[recession_start,recession_bottom]]
    non_uni_town_data = non_uni_town_data.loc[:,[recession_start,recession_bottom]]
    
    uni_town_data['price_ratio'] = uni_town_data[recession_bottom] - uni_town_data[recession_start]    
    non_uni_town_data['price_ratio'] = non_uni_town_data[recession_bottom] - non_uni_town_data[recession_start]

    st, p = ttest_ind(uni_town_data['price_ratio'], non_uni_town_data['price_ratio'], nan_policy='omit',)

    
    # get different and better values
    different = False
    if p < 0.01:
        different = True

    # determine which type of town is better
    better = ""
    if uni_town_data['price_ratio'].mean() > non_uni_town_data['price_ratio'].mean():
        better = "university town"
    else:
        better = "non-university town"

    return (different, p, better)

    
run_ttest()

(True, 0.004325214853459962, 'university town')